In [9]:
import os
import torch
import torchvision
import torchvision.transforms.functional as F
from glob import glob
from PIL.Image import open as im_open
import subprocess
import json
import pandas as pd
import sys
import pathlib
import numpy as np

In [10]:
mode = "train"
kind = "modern"
assert kind in ["modern", "old"]

def crop_bbox(annotation, crop_x, crop_y, crop_width, crop_height):
    class_id, center_x, center_y, width, height = annotation
    x_ok = crop_x <= center_x - width / 2 <= crop_x + crop_width
    y_ok = crop_y <= center_y <= crop_y + crop_height
    if (x_ok) and (y_ok):
        center_x_new = (center_x - crop_x) / crop_width
        center_y_new = (center_y - crop_y) / crop_height
        width_new = width / crop_width
        height_new = height / crop_height
        
        x_new = max(0, center_x_new - width_new / 2)
        y_new = max(0, center_y_new - height_new / 2)
        max_x_new = min(1, x_new + width_new)
        max_y_new = min(1, y_new + height_new)
        width_new = max_x_new - x_new
        height_new = max_y_new - y_new
        center_x_new = x_new + width_new / 2
        center_y_new = y_new + height_new / 2
        
        return [class_id, center_x_new, center_y_new, width_new, height_new]
    else:
        return None
    
def process_crop(filename: str):
    image_path = pathlib.Path("/workspace") / mode / kind / "images" / f"{filename}.jpg"
    label_path = pathlib.Path("/workspace") / mode / kind / "labels" / f"{filename}.txt"
    results = []
    is_important_image = False
    with open(label_path, "r") as f:
        img = im_open(image_path).convert("RGB")
        imgwidth, imgheight = img.size
        crop_width = int(imgwidth * 0.7)
        crop_height = int(imgheight * 0.7)
        crop_x = int(imgwidth * 0.25 * abs(np.random.randn()))
        crop_y = int(imgheight * 0.25 * abs(np.random.randn()))        
        for line in f.read().split("\n"):
            line = line.split(" ")
            if len(line) != 5:
                continue
            class_id, center_x, center_y, width, height = map(float, line)
            importants = [7, 8] if mode == "modern" else [4, 5]
            if class_id in importants:
                is_important_image = True
            center_x *= imgwidth
            width *= imgwidth   
            center_y *= imgheight         
            height *= imgheight            
            print((class_id, center_x, center_y, width, height), crop_x, crop_y, crop_width, crop_height)
            bbox = crop_bbox((class_id, center_x, center_y, width, height), crop_x, crop_y, crop_width, crop_height)
            if bbox is None:
                continue
            # 正規化済み
            results.append(bbox)
    if not is_important_image:
        return
    img = F.crop(img, left=crop_x, top=crop_y, width=crop_width, height=crop_height)
    img.save(image_path.with_name(f"1{filename}.jpg"))
    with open(label_path.with_name(f"1{filename}.txt"), "w") as f:
        for result in results:
            result[0] = int(result[0])
            result = map(str, result)            
            data = " ".join(result)
            f.write(data)
            f.write("\n")

for path in list((pathlib.Path("/workspace") / mode / kind / "images").iterdir()):
    print("path", path)
    p = path.resolve().as_posix()
    filename = p.split("/")[-1].split(".")[0]
    process_crop(filename)

path /workspace/train/modern/images/000001.jpg
(1.0, 921.0, 637.0, 1716.0, 1146.0) 4 503 1320 1319
(4.0, 505.0, 632.0, 208.0, 812.0) 4 503 1320 1319
(7.0, 634.0, 296.5, 14.0, 101.0) 4 503 1320 1319
(7.0, 639.0, 418.5, 14.0, 101.0) 4 503 1320 1319
path /workspace/train/modern/images/000002.jpg
(4.0, 491.00000000000006, 635.5, 518.0, 799.0) 53 100 1320 1319
(7.0, 702.0, 672.0, 28.0, 126.0) 53 100 1320 1319
(7.0, 705.5, 801.5, 13.0, 25.0) 53 100 1320 1319
(7.0, 694.0, 802.5, 12.0, 29.0) 53 100 1320 1319
(7.0, 775.0, 305.5, 26.0, 101.0) 53 100 1320 1319
(7.0, 777.5, 426.0, 21.0, 106.0) 53 100 1320 1319
(1.0, 911.0, 636.0, 1706.0, 1128.0) 53 100 1320 1319
path /workspace/train/modern/images/000003.jpg
(4.0, 501.0, 384.0, 592.0, 418.0) 7 103 1313 1313
(4.0, 497.0, 904.5, 570.0, 423.0) 7 103 1313 1313
(7.0, 193.0, 473.0, 18.0, 202.0) 7 103 1313 1313
(7.0, 204.5, 985.0000000000001, 15.0, 266.0) 7 103 1313 1313
(1.0, 920.0, 643.0, 1692.0, 1196.0) 7 103 1313 1313
path /workspace/train/modern/ima